## Connect Goggle rive where data set is placed

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Step 1: Install the required libraries.

In [2]:
! pip install tensorflow keras transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orbax-checkpoint 0.3.2 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.


## Import Libraries

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from transformers import BertTokenizer
import tensorflow as tf


FIRST TRY ::: *With* 4 sessions in 4 folds and 1 session in 1 fold  # this code use bert as features extractors and insert them directly in the lstm layers ...

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import numpy as np
from transformers import BertTokenizer, TFBertModel
from keras.models import Model
from keras.layers import Input, Dropout
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Load dataset and preprocess as needed
data = pd.read_csv('/content/drive/MyDrive/Android/audio_txt.csv').dropna(subset=['cleaned_transcript'])

MAX_SEQUENCE_LENGTH = 200
NUM_CLASSES = 2

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("distilbert-base-multilingual-cased")
bert_model = TFBertModel.from_pretrained('distilbert-base-multilingual-cased')

# Tokenize and encode your data
input_sequences = tokenizer(data['cleaned_transcript'].tolist(), padding='max_length', truncation=True, max_length=MAX_SEQUENCE_LENGTH, return_tensors="tf")
input_ids = input_sequences['input_ids']
attention_masks = input_sequences['attention_mask']

embeddings = bert_model(input_ids, attention_mask=attention_masks)[0].numpy()

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['label'])
labels = to_categorical(labels, num_classes=NUM_CLASSES)

# K-fold cross-validation
class SessionKFold:
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y=None, groups=None):
        unique_sessions = np.unique(groups)
        if self.shuffle:
            np.random.shuffle(unique_sessions)
        for i in range(self.n_splits):
            test_sessions = [unique_sessions[i]]
            train_sessions = [s for s in unique_sessions if s not in test_sessions]
            yield np.where(np.isin(groups, train_sessions))[0], np.where(np.isin(groups, test_sessions))[0]

kf = SessionKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
frame_level_accuracies = []

for train_index, val_index in kf.split(embeddings, groups=data['session_number']):
    print(f"Fold {fold}")

    # Split data
    x_train, x_val = embeddings[train_index], embeddings[val_index]
    y_train, y_val = labels[train_index], labels[val_index]

    # Define LSTM-based model
    input_layer = Input(shape=(MAX_SEQUENCE_LENGTH, 768))
    lstm_output = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2))(input_layer)
    lstm_output = Bidirectional(LSTM(64, dropout=0.2))(lstm_output)
    output_layer = Dense(NUM_CLASSES, activation='softmax')(lstm_output)
    model = Model(inputs=input_layer, outputs=output_layer)

    optimizer = Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5, batch_size=2, validation_split=0.2)

    y_pred = model.predict(x_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_val_classes = np.argmax(y_val, axis=1)

    speaker_level_accuracy = accuracy_score(y_val_classes, y_pred_classes)
    frame_level_accuracies.append(speaker_level_accuracy)

    fold += 1

avg_speaker_level_accuracy = np.mean(frame_level_accuracies)
print(f"Average Speaker Level Accuracy: {avg_speaker_level_accuracy:.4f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.transformer.layer.3.ffn.lin1.weight', 'distilbert.transformer.layer.4.output_layer_norm.weight', 'distilbert.transformer.layer.4.sa_layer_norm.bias', 'distilbert.transformer.layer.2.attention.q_lin.weight', 'distilbert.transformer.layer.2.attention.v_lin.weight', 'distilbert.transformer.layer.4.ffn.lin2.bias', 'distilbert.transformer.layer.3.sa_layer_norm.bias', 'distilbert.transformer.layer.2.attention.out_lin.bias', 'distilbert.transformer.layer.4.attention.v_lin.weight', 'distilbert.transformer.layer.1.sa_layer_norm.bias', 'distilbert.transformer.layer.2.attention.k_lin.weight', 'distilbert.transformer.layer.2.output_layer_norm.weight', 'distilbert.transformer.layer.4.f

second TRY ::: *With* 4 sessions in 4 folds and 1 session in 1 fold  # this code use bert as features extractors and bert as a model ...

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
import numpy as np
from transformers import BertTokenizer, TFBertModel
from keras.models import Model
from keras.layers import Input, Dropout
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Load your Italian dataset and preprocess as needed
data = pd.read_csv('/content/drive/MyDrive/Android/audio_txt.csv').dropna(subset=['cleaned_transcript'])

# Preprocess data
MAX_SEQUENCE_LENGTH = 200#512 I reduce this it should be 512 to run the code only
EMBEDDING_DIM = 300
NUM_CLASSES = 2

# Tokenize using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')



# Tokenize and pad sequences
input_sequences = tokenizer(data['cleaned_transcript'].tolist(), padding='max_length', truncation=True, max_length=MAX_SEQUENCE_LENGTH)
padded_sequences = np.array(input_sequences['input_ids'])
attention_masks = np.array(input_sequences["attention_mask"])



# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['label'])
labels = to_categorical(labels, num_classes=NUM_CLASSES)

# Initialize k-fold cross-validation with a custom iterator
class SessionKFold:
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y=None, groups=None):
        unique_sessions = np.unique(groups)
        if self.shuffle:
            np.random.shuffle(unique_sessions)
        for i in range(self.n_splits):
            test_sessions = [unique_sessions[i]]
            train_sessions = [s for s in unique_sessions if s not in test_sessions]
            yield np.where(np.isin(groups, train_sessions))[0], np.where(np.isin(groups, test_sessions))[0]

# Initialize your data and labels
# padded_sequences = ...
# labels = ...
# session_numbers = ...

# Initialize k-fold cross-validation
kf = SessionKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

# Initialize variables to store results
frame_level_accuracies = []

for train_index, val_index in kf.split(padded_sequences, groups=data['session_number']):
    print(f"Fold {fold}")

    # Split data into train and validation sets
    x_train, x_val = padded_sequences[train_index], padded_sequences[val_index]
    y_train, y_val = labels[train_index], labels[val_index]
    train_mask, val_mask = attention_masks[train_index], attention_masks[val_index]

    print(x_train.shape, y_train.shape)
    print(x_val.shape, y_val.shape)

    # Define the model
    input_tokens = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name="input_tokens")
    input_masks = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name="input_masks")
    bert_output = bert_model([input_tokens, input_masks])[0]
    lstm_output = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2))(bert_output)
    lstm_output = Bidirectional(LSTM(64, dropout=0.2))(lstm_output)
    output_layer = Dense(NUM_CLASSES, activation='softmax')(lstm_output)

    model = Model(inputs=[input_tokens, input_masks], outputs=output_layer)

    # Compile the model with a lower learning rate
    optimizer = Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Implement learning rate schedule
    def lr_schedule(epoch):
        return 0.0001 * (0.9 ** epoch)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    model.fit([x_train, train_mask], y_train, epochs=5, batch_size=8)

    # Make predictions on validation data
    y_pred = model.predict([x_val, val_mask])
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_val_classes = np.argmax(y_val, axis=1)

    # Calculate accuracy at full_speaker_id level
    speaker_level_accuracy = accuracy_score(y_val_classes, y_pred_classes)

    frame_level_accuracies.append(speaker_level_accuracy)

    fold += 1

# Calculate average accuracy at full_speaker_id level
avg_speaker_level_accuracy = np.mean(frame_level_accuracies)

print(f"Average Speaker Level Accuracy: {avg_speaker_level_accuracy:.4f}")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Fold 1
(719, 200) (719, 2)
(153, 200) (153, 2)
Epoch 1/5


90/90 [==============================] - 58s 418ms/step - loss: 0.6020 - accuracy: 0.7079
Epoch 2/5
90/90 [==============================] - 37s 410ms/step - loss: 0.5793 - accuracy: 0.7177
Epoch 3/5
90/90 [==============================] - 37s 412ms/step - loss: 0.5916 - accuracy: 0.7302
Epoch 4/5
90/90 [==============================] - 37s 413ms/step - loss: 0.5927 - accuracy: 0.7191
Epoch 5/5
5/5 [==============================] - 7s 414ms/step
Fold 2
(670, 200) (670, 2)
(202, 200) (202, 2)
Epoch 1/5


84/84 [==============================] - 57s 415ms/step - loss: 0.6890 - accuracy: 0.6090
Epoch 2/5
84/84 [==============================] - 35s 412ms/step - loss: 0.6691 - accuracy: 0.6119
Epoch 3/5
84/84 [==============================] - 34s 410ms/step - loss: 0.6683 - accuracy: 0.6313
Epoch 4/5
84/84 [==============================] - 35s 412ms/step - loss: 0.6655 - accuracy: 0.6134
Epoch 5/5
7/7 [==============================] - 7s 386ms/step
Fold 3
(659, 200) (659, 2)
(213, 200) (213, 2)
Epoch 1/5


83/83 [==============================] - 56s 413ms/step - loss: 0.6634 - accuracy: 0.6131
Epoch 2/5
83/83 [==============================] - 34s 410ms/step - loss: 0.6726 - accuracy: 0.6085
Epoch 3/5
83/83 [==============================] - 34s 409ms/step - loss: 0.6604 - accuracy: 0.6206
Epoch 4/5
83/83 [==============================] - 34s 410ms/step - loss: 0.6659 - accuracy: 0.6282
Epoch 5/5
7/7 [==============================] - 7s 407ms/step
Fold 4
(741, 200) (741, 2)
(131, 200) (131, 2)
Epoch 1/5


93/93 [==============================] - 60s 413ms/step - loss: 0.6555 - accuracy: 0.6194
Epoch 2/5
93/93 [==============================] - 38s 410ms/step - loss: 0.6661 - accuracy: 0.6248
Epoch 3/5
93/93 [==============================] - 38s 410ms/step - loss: 0.6563 - accuracy: 0.6424
Epoch 4/5
93/93 [==============================] - 38s 412ms/step - loss: 0.6537 - accuracy: 0.6464
Epoch 5/5
5/5 [==============================] - 6s 339ms/step
Fold 5
(699, 200) (699, 2)
(173, 200) (173, 2)
Epoch 1/5


88/88 [==============================] - 59s 413ms/step - loss: 0.6765 - accuracy: 0.5937
Epoch 2/5
88/88 [==============================] - 36s 410ms/step - loss: 0.6770 - accuracy: 0.6123
Epoch 3/5
88/88 [==============================] - 36s 409ms/step - loss: 0.6772 - accuracy: 0.6094
Epoch 4/5
88/88 [==============================] - 36s 411ms/step - loss: 0.6707 - accuracy: 0.6152
Epoch 5/5
6/6 [==============================] - 6s 383ms/step
Average Speaker Level Accuracy: 0.6387


third TRY ::: this code use 5-fold validation, bert as features extractors, then one embedding layer them BLSTM

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
import numpy as np
from transformers import BertTokenizer, TFBertModel
from keras.models import Model
from keras.layers import Input, Dropout

# Load your Italian dataset and preprocess as needed
data = pd.read_csv('/content/drive/MyDrive/Android/audio_txt.csv').dropna(subset=['cleaned_transcript'])

# Preprocess data
MAX_SEQUENCE_LENGTH = 512
EMBEDDING_DIM = 300
NUM_CLASSES = 2

# Tokenize using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Tokenize and pad sequences
input_sequences = tokenizer(data['cleaned_transcript'].tolist(), padding='max_length', truncation=True, max_length=MAX_SEQUENCE_LENGTH)
padded_sequences = np.array(input_sequences['input_ids'])

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['label'])
labels = to_categorical(labels, num_classes=NUM_CLASSES)

# Initialize k-fold cross-validation with a custom iterator
class SessionKFold:
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def split(self, X, y=None, groups=None):
        unique_sessions = np.unique(groups)
        if self.shuffle:
            np.random.shuffle(unique_sessions)
        for i in range(self.n_splits):
            test_sessions = [unique_sessions[i]]
            train_sessions = [s for s in unique_sessions if s not in test_sessions]
            yield np.where(np.isin(groups, train_sessions))[0], np.where(np.isin(groups, test_sessions))[0]

# Initialize your data and labels
# padded_sequences = ...
# labels = ...
# session_numbers = ...

# Initialize k-fold cross-validation
kf = SessionKFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

# Initialize variables to store results
frame_level_accuracies = []

# Loop through each fold
for train_index, val_index in kf.split(padded_sequences, groups=data['session_number']):
    print(f"Fold {fold}")

    # Split data into train and validation sets
    x_train, x_val = padded_sequences[train_index], padded_sequences[val_index]
    y_train, y_val = labels[train_index], labels[val_index]

    print("Training session numbers:", data['session_number'].iloc[train_index].values)
    print("Validation session numbers:", data['session_number'].iloc[val_index].values)
    print("------------------------------")

    # Build the LSTM model with Bidirectional layers
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.vocab), output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(NUM_CLASSES, activation='softmax'))


    # Compile the model with a lower learning rate
    optimizer = Adam(learning_rate=0.0005)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Implement learning rate schedule
    def lr_schedule(epoch):
        return 0.0005 * (0.9 ** epoch)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    # Implement early stopping
    early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

    # Train the model
    model.fit(x_train, y_train, epochs=5, batch_size=8, validation_split=0.2, callbacks=[lr_scheduler, early_stopping])

    # Make predictions on validation data
    y_pred = model.predict(x_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_val_classes = np.argmax(y_val, axis=1)

    # Calculate accuracy at full_speaker_id level
    speaker_level_accuracy = accuracy_score(y_val_classes, y_pred_classes)

    frame_level_accuracies.append(speaker_level_accuracy)

    fold += 1

# Calculate average accuracy at full_speaker_id level
avg_speaker_level_accuracy = np.mean(frame_level_accuracies)

print(f"Average Speaker Level Accuracy: {avg_speaker_level_accuracy:.4f}")


Fold 1
Training session numbers: [4 4 4 4 4 4 4 5 5 2 2 2 2 2 2 2 5 5 5 5 5 5 5 5 5 3 3 3 4 4 4 4 4 4 4 5 5
 5 2 2 2 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4
 4 4 4 4 4 3 3 3 3 3 3 3 3 3 5 5 5 5 5 5 5 5 5 4 4 4 4 4 3 3 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 3 3 3 3 3 3 3 5 5 5 5 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 4 4 4 4 4 4
 4 3 3 3 3 5 5 5 5 5 5 4 4 4 4 4 4 4 2 2 2 2 2 2 4 4 4 4 4 4 4 3 3 3 4 3 3
 5 5 5 5 5 5 5 5 5 5 5 5 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5
 5 5 5 5 5 5 5 5 5 5 5 2 2 2 2 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 4 4 4 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 4 4 4 4 4 5 5 5 5 5 5 5 5 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 

Epoch 1/5
47/74 [==================>...........] - ETA: 1:27 - loss: 0.6627 - accuracy: 0.6197

KeyboardInterrupt: ignored